### AR6 CSV Fixer

#### This part un-melds the data into 'wide' format

In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv("/net/home/h04/gmunday/Desktop/CSSPBrazil/FaIR/AR6Ilusplus_RCMIPhist.csv")
df.drop(columns='Unnamed: 0', inplace=True)
df_p = df.set_index(['Model', 'Scenario', 'Region', 'Variable', 'Unit', 'Year'])['value'].unstack().reset_index()
df_p.sort_values('Model', inplace=True)
df_p = df_p.rename_axis(None, axis=1).reset_index()
df_p.drop(columns='index', inplace=True)

### Removing Historical Fields that don't match up with AR6 

In [6]:
removals = ['AR6 climate diagnostics|Infilled|', 'F-Gases|', 'Montreal Gases|CFC|', 'Montreal Gases|']
hfc_changes = ['HFC245fa', 'HFC4310mee']
hfc_correct = ['HFC245ca', 'HFC43-10']
co2_changes = ['MAGICC AFOLU', 'MAGICC Fossil and Industrial']
co2_correct = ['AFOLU', 'Energy and Industrial Processes']

df_p['Variable'] = df_p['Variable'].astype(str)
for y in removals:
    df_p['Variable'] = df_p['Variable'].apply(lambda x: x.replace(y, ''))   
    
for i, y in enumerate(hfc_changes):
    df_p['Variable'] = df_p['Variable'].apply(lambda x: x.replace(y, hfc_correct[i]))

for i, y in enumerate(co2_changes):
    df_p['Variable'] = df_p['Variable'].apply(lambda x: x.replace(y, co2_correct[i]))

df_AR6 = df_p.loc[df_p['Model'] == 'AIM/CGE 2.2']

In [7]:
# isolating the extra variables we don't want and removing them
df_badvars = df_p[~df_p.Variable.isin(df_AR6.Variable)]
df_new = df_p[~df_p.Variable.isin(df_badvars.Variable)]
df_new = df_new.drop(df_new.index[df_new['Region'] != 'World'])

In [8]:
if len(df_new) == len(df_p):
    print("Didn't work")
else:
    print(f"Amount dropped = {abs(len(df_new) - len(df_p))}")

Amount dropped = 750


In [9]:
hist = df_new.loc[df_new['Model'] == 'CEDS/UVA/GCP/PRIMAP']

x = [x for x in hist.Variable]
y = [x for x in df_AR6.Variable]

##### Checking for missing variables

In [10]:
unique = [k for k in y if k not in x]

print(f'Number of historical vars missing: {len(unique)}')
print(f'\nVars missing: \n{unique}')

Number of historical vars missing: 0

Vars missing: 
[]


##### Dropping missing variables from the rest of the dataframe (CHECK)

In [11]:
for x in unique:
    df_new = df_new.drop(df_new.index[df_new['Variable'] == x])
    
hist = df_new.loc[df_new['Model'] == 'CEDS/UVA/GCP/PRIMAP']
x = [x for x in hist.Variable]
y = [x for x in df_new.Variable]

unique = [k for k in y if k not in x]
unique_1 = [k for k in x if k not in y]

print(unique_1)
print(f'Number of historical vars missing: {len(unique)}')
print(f'\nVars missing: \n{unique}')

[]
Number of historical vars missing: 0

Vars missing: 
[]


In [12]:
# here we need to combine the CO2 values for each scenario
# aggregate_functions = {x : 'sum' for x in range(1750, 2101)}
# df_new = df_new.groupby(['Model', 'Scenario', 'Region', 'Variable', 'Unit'], as_index=False).aggregate(aggregate_functions)

Check there are equal amounts of vars per scenario:

In [13]:
counts = df_new['Scenario'].value_counts()
for i in range(1, len(counts)):
    if counts[i] == counts[i-1]:
        continue
    else:
        raise ValueError('Scenarios have differing amounts of variables.')

print(f'All scenarios have equal amounts of variables: \n\n{counts}')

All scenarios have equal amounts of variables: 

EN_NPi2020_900f                  52
historical                       52
EN_NPi2020_400f_lowBECCS         52
EN_INDCi2030_500f                52
LowEnergyDemand_1.3_IPCC         52
SSP2_openres_lc_50               52
SusDev_SDP-PkBudg1000            52
DeepElec_SSP2_ HighRE_Budg900    52
CO_Bridge                        52
Name: Scenario, dtype: int64


Now we need to tile the historical data over each scenario from 1750 - 2014..

In [14]:
# order alphabetically
df_new = df_new.sort_values(['Scenario', 'Variable'], ascending=[True, True])
hist = df_new.loc[df_new['Model'] == 'CEDS/UVA/GCP/PRIMAP']

# get the historical data per year
for yr in range(1750, 2015):
    hist_data = hist[yr]
    hist_tiled = np.tile(hist_data, len(df_new['Scenario'].value_counts()))
    df_new[yr] = hist_tiled
    if len(df_new[yr]) == len(df_new['Model']):
        continue
    else:
        ValueError('Tiling did not work properly.')


Now delete historical-data-only rows:

In [15]:
df_new = df_new.drop(df_new.index[df_new['Scenario'] == 'historical'])

if 'historical' not in df_new['Scenario']:
    print('Great success!')

Great success!


##### Finally, read to CSV

In [16]:
df_new.to_csv('AR6Illus_RCMIP.csv')